In [2]:
import pandas as pd
import duckdb

C:\Users\Lione\AppData\Local\Temp\ipykernel_10080\2256573232.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
# EQUIPO CON MAYOR CANTIDAD DE PARTIDOS GANADOS

# primero consigo una tabla con todos los partidos que jugó un equipo de local estos años
# y luego de esos partidos cuento cuantos ganó
ganados_local = duckdb.sql('''
                           SELECT ID_equipo_local, Resultado, COUNT(*) AS Ganados_local
                           FROM (
                                    SELECT ID_local AS ID_equipo_local, 
                                    CASE WHEN Goles_local > Goles_visitante THEN 'Ganado'
                                    WHEN Goles_visitante > Goles_local THEN 'Perdido'
                                    ELSE 'Empatado'
                                    END AS Resultado
                                    FROM tabla_partidos.csv
                                    WHERE ID_liga = 24558 AND CAST(TEMPORADA AS TEXT) IN ('2012/2013','2013/2014','2014/2015','2015/2016')
                                )
                           GROUP BY ID_equipo_local, Resultado
                           HAVING Resultado = 'Ganado'
                           ORDER BY ID_equipo_local
                           ''')
# hago lo mismo para visitantes
ganados_visitante = duckdb.sql('''
                           SELECT ID_equipo_visitante, Resultado, COUNT(*) AS Ganados_visitante
                           FROM (
                                    SELECT ID_visitante AS ID_equipo_visitante, 
                                    CASE WHEN Goles_local > Goles_visitante THEN 'Perdido'
                                    WHEN Goles_visitante > Goles_local THEN 'Ganado'
                                    ELSE 'Empatado'
                                    END AS Resultado
                                    FROM tabla_partidos.csv
                                    WHERE ID_liga = 24558 AND CAST(TEMPORADA AS TEXT) IN ('2012/2013','2013/2014','2014/2015','2015/2016')
                                )
                           GROUP BY ID_equipo_visitante, Resultado
                           HAVING Resultado = 'Ganado'
                           ORDER BY ID_equipo_visitante
                           ''')

# ahora sumo la cantidad total de partidos ganados por los equipos: locales + visitante
ganados_totales = duckdb.sql('''
                       SELECT ID_equipo_local AS ID_equipo, Ganados_local + Ganados_visitante AS GANADOS
                       FROM ganados_local
                       INNER JOIN ganados_visitante 
                       ON ID_equipo_local = ID_equipo_visitante
                       ORDER BY GANADOS
                       ''')

# luego en esos partidos tengo que buscar el que mas ganó, así que necesito esta subquery
equipo_mas_ganador = duckdb.sql('''
                          SELECT ID_equipo
                          FROM ganados_totales
                          WHERE GANADOS = (SELECT MAX(GANADOS) AS MAXGANADOS
                                            FROM ganados_totales)
                          ''')

equipo_mas_ganador

┌───────────┐
│ ID_equipo │
│   int64   │
├───────────┤
│      9931 │
└───────────┘

In [4]:
#EQUIPO CON MAYOR CANTIDAD DE PARTIDOS PERDIDOS DE CADA AÑO

perdidos_local_por_año = duckdb.sql('''
                                    SELECT ID_local, Resultado, Temporada, COUNT(*) AS Perdidos_local
                                    FROM (
                                        SELECT ID_local, 
                                        CASE WHEN Goles_local > Goles_visitante THEN 'Ganado'
                                        WHEN Goles_visitante > Goles_local THEN 'Perdido'
                                        ELSE 'Empatado'
                                        END AS Resultado, Temporada
                                        FROM tabla_partidos.csv
                                        WHERE ID_liga = 24558 
                                        AND CAST(Temporada AS TEXT) IN ('2012/2013','2013/2014','2014/2015','2015/2016')
                                    )
                                    GROUP BY ID_local, Resultado, Temporada
                                    HAVING Resultado = 'Perdido'
                                    ORDER BY ID_local

                                    ''')
perdidos_visitante_por_año = duckdb.sql('''
                                    SELECT ID_visitante, Resultado, Temporada, COUNT(*) AS Perdidos_visitante
                                    FROM (
                                        SELECT ID_visitante, 
                                        CASE WHEN Goles_local > Goles_visitante THEN 'Perdido'
                                        WHEN Goles_visitante > Goles_local THEN 'Ganado'
                                        ELSE 'Empatado'
                                        END AS Resultado, Temporada
                                        FROM tabla_partidos.csv
                                        WHERE ID_liga = 24558 
                                        AND CAST(Temporada AS TEXT) IN ('2012/2013','2013/2014','2014/2015','2015/2016')
                                    )
                                    GROUP BY ID_visitante, Resultado, Temporada
                                    HAVING Resultado = 'Perdido'
                                    ORDER BY ID_visitante

                                    ''')

perdidos_totales_por_año = duckdb.sql('''
                                      SELECT ID_local AS ID_equipo, perloaño.Temporada, Perdidos_local + Perdidos_visitante AS Perdidos_totales
                                      FROM perdidos_local_por_año AS perloaño
                                      INNER JOIN perdidos_visitante_por_año AS perviaño
                                      ON ID_local = ID_visitante AND perloaño.Temporada = perviaño.Temporada
                                      ORDER BY ID_local
                                      ''')


mayores_perdedores = duckdb.sql('''
                                SELECT perdidos1.ID_equipo, perdidos1.Temporada, perdidos1.Perdidos_totales
                                FROM perdidos_totales_por_año AS perdidos1
                                WHERE perdidos1.Perdidos_totales >= ALL  (
                                    SELECT perdidos2.Perdidos_totales
                                    FROM perdidos_totales_por_año AS perdidos2
                                    WHERE perdidos2.Temporada = perdidos1.Temporada
                                
                                )
                                
                                ''')
mayores_perdedores

┌───────────┬───────────┬──────────────────┐
│ ID_equipo │ Temporada │ Perdidos_totales │
│   int64   │  varchar  │      int64       │
├───────────┼───────────┼──────────────────┤
│      7730 │ 2013/2014 │               26 │
│      7896 │ 2015/2016 │               19 │
│      9777 │ 2012/2013 │               22 │
│      9824 │ 2014/2015 │               19 │
└───────────┴───────────┴──────────────────┘

In [6]:
# Equipo con mayor cantidad de partidos empatados en el último año: 2015/2016

empatados_local = duckdb.sql('''
                                    SELECT ID_local, Resultado, COUNT(*) AS Empatados_local
                                    FROM (
                                        SELECT ID_local, 
                                        CASE WHEN Goles_local > Goles_visitante THEN 'Ganado'
                                        WHEN Goles_visitante > Goles_local THEN 'Perdido'
                                        ELSE 'Empatado'
                                        END AS Resultado, Temporada
                                        FROM tabla_partidos.csv
                                        WHERE ID_liga = 24558 
                                        AND CAST(Temporada AS TEXT) = '2015/2016'
                                    )
                                    GROUP BY ID_local, Resultado, Temporada
                                    HAVING Resultado = 'Empatado'
                                    ORDER BY ID_local

                                    ''')
empatados_visitante = duckdb.sql('''
                                 SELECT ID_visitante, Resultado, COUNT(*) AS Empatados_visitante
                                    FROM (
                                        SELECT ID_visitante, 
                                        CASE WHEN Goles_local > Goles_visitante THEN 'Ganado'
                                        WHEN Goles_visitante > Goles_local THEN 'Perdido'
                                        ELSE 'Empatado'
                                        END AS Resultado, Temporada
                                        FROM tabla_partidos.csv
                                        WHERE ID_liga = 24558 
                                        AND CAST(Temporada AS TEXT) = '2015/2016'
                                    )
                                    GROUP BY ID_visitante, Resultado, Temporada
                                    HAVING Resultado = 'Empatado'
                                    ORDER BY ID_visitante
                                 ''')
empatados_totales = duckdb.sql('''
                               SELECT ID_local AS ID_equipo, Empatados_local + Empatados_visitante AS Empatados
                               FROM empatados_local
                               INNER JOIN empatados_visitante
                               ON ID_local = ID_visitante
                               
                               ''')

equipo_mas_empatador = duckdb.sql('''
                                  SELECT ID_equipo
                                  FROM empatados_totales
                                  WHERE Empatados = (
                                      SELECT MAX(Empatados) AS elmaximo
                                      FROM empatados_totales
                                  )
                                  ''')
equipo_mas_empatador

┌───────────┐
│ ID_equipo │
│   int64   │
├───────────┤
│      9824 │
└───────────┘

In [7]:
# Cual es el jugador que más goles convirtió?
# basicamente hay que contar la cantidad de veces que aparece cada jugador

goleador = duckdb.sql('''
                      SELECT ID_Jugador, COUNT(*) AS Goles
                      FROM (
                          SELECT *
                          FROM tabla_goles.csv AS g
                          WHERE g.ID_Partido IN (
                              SELECT ID_Partido
                              FROM tabla_partidos.csv
                              WHERE ID_liga = 24558 
                              AND CAST(Temporada AS TEXT) IN ('2012/2013','2013/2014','2014/2015','2015/2016')
                          )
                      )
                      GROUP BY ID_Jugador
                      ''')
# no me creo que haya funcionado hacer el in con el where
maximo_goleador = duckdb.sql('''
                             SELECT ID_Jugador, Goles
                             FROM goleador
                             WHERE Goles = (
                                 SELECT MAX(Goles) AS maximo
                                 FROM goleador
                             )
                             ''')
maximo_goleador

┌────────────┬───────┐
│ ID_Jugador │ Goles │
│   int64    │ int64 │
├────────────┼───────┤
│      34082 │     6 │
│      40274 │     6 │
└────────────┴───────┘